In [127]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import random
import math

import pandas as pd 
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA = torch.cuda.is_available()

# 设置随机数的seed，这样保证每次测试的数据一致
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)



# 参数初始化





In [128]:
# 设定一些初始hyper parameters(超参数)
C = 3 # context window # 正样本的取值返回，就是在目标前后C个词以内的词认为是正样本
#K代表负采样数量，当有一个正样本是，对应要取多少个负样本：样本小时K取大一些(5-20或更大)，当样本大时K取小一些（2-5）
K = 100 # number of negetive samples:
NUM_EPOCHS = 2
MAX_VOCAB_SIZE = 30000 # One-Hot的最length
BATCH_SIZE = 128 
LEARNING_RATE = 0.2
EMBEDDING_SIZE = 100 # 词嵌入向量的维度

def word_tokenize(text):
    return text.split()






# 数据预处理








In [130]:
# 从绝对路径中读取文件
with open("/Users/zhenwuzhou/.keras/datasets/text8/text8.train.txt","r") as fin:
    text = fin.read()

# print(text[:1000]) # 取前1000个字符
text = text.split() # 把Text变为List
text[:5] 

['anarchism', 'originated', 'as', 'a', 'term']

In [131]:
# 创建one-hot词表征字典
# 将出现频率最高的前MAX_VOCAB_SIZE - 1词加入到字典中去；
# vocab["of"]:537144;其中key是单词，value是单词出现的次数
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE - 1)) 
# 把<unk>这个代表位置的词加入到字典中去：
# 其中vocab["<unk>"]：617240；key是"<unk>"，value是text的总长度减去所有加入到字典中的词的总数，
# 这个值时一个>=0的值：0说明所有的词都加入到字典中了，>0说明还有没有加入到字典中的词，我们用"<unk>"表示
# 实际中的话数字这种东西可以用<nums>来代替
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))

# index_to_word 是把所有加入到字典中的不重复的词整理成list
index_to_word = [word for word in vocab.keys()]
# word_to_index 是吧每个词按照位置进行编号：
#word_to_index["<unk>"]:29999 可以代表的是词，value是词的one-hot索引位置
word_to_index = {word:i for i, word in enumerate(index_to_word)}

# 真实的不重复的词向量总共有多少个：VOCAB_SIZE：VOCAB_SIZE
VOCAB_SIZE = len(index_to_word)

# list(word_to_index)[:100] 可以取出字典的前100个看看

In [132]:
# 为了后续的随机采样，我们需要知道每个词出现的frequence(频率)
# 我们先统计出每个词出现的次数，其实就是字典vocab中的每个词的values，这里要把数据类型变为float32
# 注意这里也把"<unk>"的频率算了进去
word_counts = np.array([count for count in vocab.values()],dtype=np.float32)
# 用每个词出现的次数/素有词出现的总次数来计算出每个词的出现频率
word_frequence = word_counts/np.sum(word_counts)

# 为了防止高频词语低频词取到的概率差距过大，让每个词的概率去做一个指数为（3/4）的指数运算
# 这个是论文作者实验得到的一种比较好的方法，但是不一定是最优的取法
word_frequence = word_frequence**(3./4.)
word_frequence = word_frequence / np.sum(word_frequence)





# Dataloader来进行数据封装处理






# 实现Dataloader
   一个dataloader需要以下内容：
   1:把所有text编码成数字
   2:保存vocabulary,单词count，normalized word frequency
   3:每个iteration sample一个中心词
   4:根据当前的中心词返回context单词
   5:根据中心词sample一些negative单词
   6:返回单词的counts
   
 这里有一个好的tutorial介绍如何使用Pytorch dataloader.为了使用dataloader，我们需要定义以下两个function：
     1: __len__ function需要返回整个数据集中有多少个item
     2: __get__ 根据给定的index返回一个item
  有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等

In [133]:
# 创建dataset和dataloader
#torch.utils.data.Dataset
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self,text,word_to_index,index_to_word,word_frequence,word_counts):
        super(WordEmbeddingDataset,self).__init__()
        # 把text 文本变成一个有每个词在字典中所有位置构成的数字化的结构文本
        # word_to_index.get(word,word_to_index["<unk>"]) 要么是这个词的位置索引，要么就是"<unk>"的索引
        # self.text_encoded 就是encoded后的整个文本数据
        self.text_encoded = [word_to_index.get(word,word_to_index["<unk>"]) for word in text]
        self.text_encoded = torch.LongTensor(self.text_encoded) # 变成longTensor
        self.word_to_index = word_to_index
        self.index_to_word = index_to_word
        self.word_frequence = torch.Tensor(word_frequence)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self): # 注意别忘记写self
        # 这个数据集一共有多少个item
        return len(self.text_encoded)
        
    def __getitem__(self,idx):
        # idx是相对于整个文本text的索引
        # 首先要取出中心词
        center_word = self.text_encoded[idx]
        # 然后要取出positive的正样本:目标索引的前后C个词
        # 首先我们先计算出前后C个词的位置索引，需要考虑左右边界超出文本长度的情况
        pos_indices = list(range(max(idx-C,0),idx))+list(range(idx+1,min(idx+C+1,len(self.text_encoded))))
#         pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        # 根据索引位置取出正样本词
        pos_words = self.text_encoded[pos_indices]
        
        #取出负样本的词:torch.multinomial方法帮我们去采样
        #K*pos_words.shape[0]表示对于每一个正样本，都对应采样K个neg_words的负样本
        #replacement=True表示是采样样本可以重复
        neg_words = torch.multinomial(self.word_frequence,K*pos_words.shape[0],replacement=True)
        
        return center_word,pos_words,neg_words

In [134]:
# index = 5 c=3的时候我想取到正样本是[2, 3, 4, 6, 7, 8]
test = list(range(2,5)) +list(range(6,9))
# 所以正样本的索引应该是：list(range(idx-C,idx))+list(range(idx+1,idx+C+1))
print(test)
# 但是如果出现前后超过文本长度怎么办呢:现在index=5，c=6；文本总长度为10
# 让左边界最小是0，右边界最大是文本总长度，因为右边不会取到本应+1，但是索引是从0开始算的，所以最终值等于len
test2 = list(range(max(-1,0),5))+list(range(6,min(12,11)))
print(test2)
## 所以最终公式为：list(range(max(idx-C,0),idx))+list(range(idx+1,min(idx+C+1,len(self.text_encoded)+1)))
# test3 = [i%10 for i in test2]
# test3

[2, 3, 4, 6, 7, 8]
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10]


In [135]:
# 创建dataset和dataloader
dataset = WordEmbeddingDataset(text,word_to_index,index_to_word,word_frequence,word_counts)
dataloader = tud.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)

In [136]:
# 取出前面100个encode后的数据
dataset.text_encoded[:100]

tensor([ 4813,  3139,    11,     5,   194,     1,  3015,    46,    59,   155,
          127,   741,   461, 10485,   133,     0, 25752,     1,     0,   108,
          833,     2,     0, 16267, 29999,     1,     0,   152,   833,  3493,
            0,   194,    10,   186,    59,     4,     5, 10620,   213,     6,
         1332,   102,   437,    19,    59,  2764,   355,     6,  3625,     0,
          709,     1,   364,    26,    40,    37,    53,   527,    97,    11,
            5,  1398,  2929,    18,   562,   691,  6644,     0,   252,  4813,
           10,  1043,    27,     0,   316,   247, 29999,  2964,   789,   189,
         4813,    11,     5,   201,   569,    10,     0,  1107,    19,  2581,
           25,  8819,     2,   273,    31,  4089,   140,    58,    25,  6494])

In [137]:
# 验证取得的正负样本是否正确，是否处理了边界问题
center_word,pos_words,neg_words = dataset.__getitem__(0)
print(pos_words)
sorted(neg_words) # 查看样本是否可以重复

tensor([3139,   11,    5])


[tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(3),
 tensor(4),
 tensor(4),
 tensor(6),
 tensor(7),
 tensor(7),
 tensor(8),
 tensor(10),
 tensor(11),
 tensor(12),
 tensor(13),
 tensor(18),
 tensor(18),
 tensor(19),
 tensor(19),
 tensor(22),
 tensor(23),
 tensor(25),
 tensor(28),
 tensor(28),
 tensor(30),
 tensor(31),
 tensor(36),
 tensor(37),
 tensor(40),
 tensor(41),
 tensor(41),
 tensor(48),
 tensor(48),
 tensor(53),
 tensor(55),
 tensor(55),
 tensor(57),
 tensor(60),
 tensor(63),
 tensor(64),
 tensor(67),
 tensor(68),
 tensor(78),
 tensor(80),
 tensor(82),
 tensor(93),
 tensor(101),
 tensor(103),
 tensor(111),
 tensor(113),
 tensor(146),
 tensor(147),
 tensor(160),
 tensor(163),
 tensor(166),
 tensor(169),
 tensor(175),
 tensor(175),
 tensor(178),
 tensor(181),
 tensor(183),
 tensor(184),
 tensor(185),
 tensor(189),
 tensor(192),
 tensor(192),
 tensor(211),
 tensor(218),
 tensor(225),
 tensor(243),
 tens

In [138]:
for i, (center_word,pos_words,neg_words) in enumerate(dataloader):
    print("center_word:",center_word,"\n",
          "pos_words:",pos_words,"\n"
          "neg_words",neg_words)
    if i > 0:
        break;

center_word: tensor([    0,  2624,     0,    41,     1,  3052,   270, 29999,    34,     9,
          354,     1,   118,  7501,   529,   532,    13,   114,     9,  3081,
            0,   632,     1,    22,     1,    28,  1644,     2,    19,  1785,
            6,     5,  1377,     5,     8,   190,     8,  2891,   204,     7,
            9,     3,     9,  4139, 10945,    14,  2299,     1,   636,  1271,
        17923,   840, 29999,     0, 29999,   786,    41,  4975,   584,  2076,
         7339,  2373,    42,     5,     2,     1,  3405,  1042,   790,  2614,
          138,     8,    21,    39,   361,   108, 29999,  1698,    17,  2252,
         9185,     8,  1463,   906,    12,  2083,     4,   133,   871,     0,
           40,     3, 18695,   171,  1639,     2,  3258,   793,   183,     0,
         1831,     3,  2834,     3,    45,     2,   343,    13,     6,   561,
        29999,    62,     4,    33, 17351,   477,  1675,     2,  3299,   512,
            1,    82, 20799,   201,  5001,   135,  






# 定义Pytroch模型






In [160]:
class EmbeddingModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super(EmbeddingModel,self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.out_embed = nn.Embedding(self.embed_size,self.embed_size)
        
        # 这里可以对初始权重进行归一化处理
#         initrange = 0.5/ self.embed_size
#         self.in_embed.weight.data.uniform_(-initrange,initrange)
        
    def forward(self,input_labels,pos_labels,neg_labels):
        # input_label: [batch_size]
        # pos_labels: [batch_size,(window_size * 2)] # 前后各区C个所以要*2，但是越界的时候不一定
        # neg_labels: [batch_size,(window_size * 2 * K)] # 每个正样本对应K个负样本
        
        input_embedding = self.in_embed(input_labels) # [batch_size,embed_size]
        pos_embedding = self.in_embed(pos_labels) # [batch_size,(window_size * 2),embed_size]
        neg_embedding = self.in_embed(neg_labels) # [batch_size,(window_size * 2 * K),embed_size]
        
        # 重中之重就是如何定义LossFunction：
        # LossFunction要满足越是我们期望的结果值越小，越是我们不期望的结果值越大
        # 我们期望目标词和正样本相似度要高：那么当计算出目标词和正样本相似度越高，代价函数值就要越小，反之越大
        # 我们不希望目标词和负样本相似度高：当计算出目标词语负样本相似度越高，代价值就要越高，反之越小
        # 我们期望目标词语正样本的相似度要高于与负样本的相似度
        
        # 第一步我们先计算相似度：相似度采用余弦相似度的简易版即为向量內积来作为相似度计算的公式
        # 因为维度不统一：input_embedding矩阵的秩为2，而pos_embedding和neg_embedding的矩阵的秩为三
        # 所以先对input_embedding做升维操作unsqueeze;squeeze是压紧，unsqueeze可以理解为展开
        input_embedding = input_embedding.unsqueeze(2) # [batch_size,embed_size,1]
        
        
        #这里要用到一个矩阵运行方法：https://pytorch.org/docs/stable/torch.html
        # torch.bmm(input, mat2, out=None)
        # If input is a (b \times n \times m)(b×n×m) tensor, 
        # mat2 is a (b \times m \times p)(b×m×p) tensor, 
        # out will be a (b \times n \times p)(b×n×p) tensor.
        #         >>> input = torch.randn(10, 3, 4)
        #         >>> mat2 = torch.randn(10, 4, 5)
        #         >>> res = torch.bmm(input, mat2)
        #         >>> res.size()
        #         torch.Size([10, 3, 5])
        
        # 计算目标词与正样本的相似度：
        # 计算完[batch_size,(window_size * 2),1],然后用squeeze把最后一维消掉
        pos_dot = torch.bmm(pos_embedding,input_embedding).squeeze(2)#[batch_size,(window_size * 2)]
        # 用SIGMOD激活后取值变为0-1就可以直接当做相似度值，但是作为loss这个值不合适
        # 所以在用一个指数函数log来去定义损失值，在x取值0-1时，logx的取值是-无穷到0
        # 对于正样本x越接近1说明越相似，logx的值越大，-logx的值越小，-logx就可以作为损失函数，
        # 本质上与sigmoid的二分类损失函数的定义还有有一定相似度的，值不过这里不在依赖标签值了(用文本出现的接近程度)
        log_pos = F.logsigmoid(pos_dot).sum(1) # 第二维度的是所有正样本，所以要在第二个维度上做加和操作
        
        # 同理计算目标词语负样本的相似度
        # 注意这里的是-input_embedding，因为正样本需要越相似越好，负样本需要越不相似越好
        neg_dot = torch.bmm(neg_embedding,-input_embedding).squeeze(2)#[batch_size,(window_size * 2 * K)]
        log_neg = F.logsigmoid(neg_dot).sum(1)
        
        loss = -(log_pos +log_neg) # 最后别忘了加上负号
        
        return loss
    
    def input_embeddings(self):
        return self.in_embed.weight.data.cpu().numpy()


In [161]:
# 定义模型
model = EmbeddingModel(VOCAB_SIZE,EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()

In [162]:
print(model.in_embed.weight.shape)
model.in_embed.weight.data.cpu().numpy()

torch.Size([30000, 100])


array([[ 0.3948246 , -1.3977673 ,  0.582277  , ...,  1.0021156 ,
         0.3033375 ,  0.9187573 ],
       [ 1.2000518 , -1.6536685 , -0.5187361 , ...,  1.5897415 ,
        -1.7625477 , -2.0025969 ],
       [-1.334339  ,  0.32504746, -0.08529827, ..., -1.4065667 ,
         0.84071934,  0.58231497],
       ...,
       [ 0.09752222, -0.23254772,  0.60725886, ..., -0.8021786 ,
        -1.4678984 , -0.80487645],
       [ 0.3784622 ,  1.4795929 ,  0.39129415, ..., -0.34258682,
         1.310756  ,  0.763132  ],
       [-2.5029325 ,  1.6724156 , -1.7058324 , ...,  1.409471  ,
        -0.16565372, -0.65119296]], dtype=float32)

In [143]:
# 定义优化器
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

# 开始进行训练
for e in range(NUM_EPOCHS):
    for i,(input_labels,pos_labels,neg_labels) in enumerate(dataloader):
#         print("center_word:",center_word,"\n",
#           "pos_words:",pos_words,"\n"
#           "neg_words",neg_words)
#         if i > 0:
#             break;
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        
        #在训练数据之前先把grad清0
        optimizer.zero_grad()
        
        # 定义loss函数
        loss = model(input_labels,pos_labels,neg_labels).mean()
        
        # 进行反向传播
        loss.backward()
        
        # 进行梯度更新
        optimizer.step()
        
        if i % 100 == 0:
            print("epoch",e,"iteration",i,loss.item())
        


epoch 0 iteration 0 2587.443603515625
epoch 0 iteration 100 1204.2564697265625


KeyboardInterrupt: 






# 下面是评估模型的代码，已经训练模型的代码 







In [163]:
embedding_weights = model.input_embeddings()
def evaluate(filename,embedding_weights):
    if filename.endswith(".csv"):
        data = pd.read_csv(filename,sep=",")
    else:
        data = pd.read_csv(filename,sep="\t")
    
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:, 0:2].index:
        word1, word2 = data.iloc[i, 0], data.iloc[i, 1]
        if word1 not in word_to_index or word2 not in word_to_index:
            continue
        else:
            word1_idx, word2_idx = word_to_index[word1],word_to_index[word2]
            # 利用训练返回的embedding_weights俩进行单词1和单词2的词向量表征
            word1_embed,word2_embed = embedding_weights[[word1_idx]],embedding_weights[[word2_idx]]
            #利用sklearn.metrics.pairwise.cosine_similarity来计算两个向量的余弦相似度
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed,word2_embed)))
            human_similarity.append(float(data.iloc[i,2]))
        
        return scipy.stats.spearmanr(human_similarity,model_similarity)
    
def find_nearest(word):
    index = word_to_index[word]
    embedding = embedding_weights[index]
    # 利用scipy.spatial.distance.cosine来找余弦相似度
    cos_dis = np.array([scipy.spatial.distance.cosine(e,embedding) for e in embedding_weights])
    return [index_to_word[i] for i in cos_dis.argsort()[:10]]

In [164]:
find_nearest("of")

['of',
 'lamp',
 'tracked',
 'broca',
 'senatorial',
 'transparency',
 'update',
 'regression',
 'forecasting',
 'lobbied']